<font size=6>**TD 2 : classification supervisée**</font>

Ce notebook est un exemple de traitement des données textuelles à des fins de classification binaires (2 classes uniquement).

In [1]:
# !pip install xgboost -q

In [2]:
# libraires utilisées

import numpy as np
import pandas as pd

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

# différents classifieurs
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import RidgeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier

In [3]:
# pour ignorer les avertissements

import warnings
warnings.filterwarnings('ignore')

# chargement des données

Chargement des données à partir d'un fichier .csv (colonnes séparées par une tabulation = "\t").

Nous allons utiliser, pour la démonstration, un jeu de données contenant des propos toxiques diffusés sur Internet (cf. défi Kaggle de 2018 : https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge).

**Attention :** certains propos tenus dans les données peuvent choquer (insultes, racisme, etc.). Si vous le préferez, rien ne vous oblige à aller lire les textes d'origine.

In [30]:
data_load = pd.read_csv("https://raw.githubusercontent.com/upunaprosk/ul2-nlp-course/refs/heads/2024/TD2/toxic_10k.csv", sep="\t")

In [31]:
data_load.shape

(10000, 3)

Affichage d'un extrait :

In [6]:
# on préfère affiche des exemples non toxiques, situés à la fin
data_load.tail()

,ids,textes,classes
9995,83838,"I'm afraid I know little about taxonomy, so ha...",0
9996,79900,"Talk:Jerusalem Hi, I would appreciate it if ...",0
9997,119587,Explanation OTRS is for the people who have ...,0
9998,86719,friends' suspicions The article says: Schiav...,0
9999,92604,"""I don't know anyone who has entertained the b...",0


In [7]:
textes = data_load["textes"]
classes = data_load["classes"]

Distribution des valeurs pour la classe :

In [8]:
unique_values, counts = np.unique(classes, return_counts=True)

for value, count in zip(unique_values, counts):
    print(f"{value} occurs {count} times")

0 occurs 5000 times
1 occurs 5000 times


# Mise en forme des données

Construction de la matrice Documents x Termes (cf. TP précédent)

## Tâche : Vectorisation de textes avec TF-IDF

L'objectif de cette tâche est de transformer un corpus de documents textuels en une matrice de caractéristiques TF-IDF. Cela consiste à utiliser le `TfidfVectorizer` de la bibliothèque `sklearn` pour mesurer l'importance des mots et des paires de mots (unigrammes et bigrammes) dans le corpus.

Documentation: https://scikit-learn.org/1.5/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

In [9]:
vectorize = TfidfVectorizer(...) # YOUR Code
vectorize.fit(textes)
data = vectorize.transform(textes)

Division du jeu de données initial en deux sous-ensembles :

- données d'entraînement ou *training set*
- données de test (ici appelé parfois valiation)

In [10]:
subtrain_X, subtest_X, subtrain_y, subtest_y = train_test_split(data, classes,
                                                    train_size=0.7,
                                                    test_size=0.3,
                                                    random_state=42,
                                                   stratify=classes)

In [11]:
# vérifions que la stratification a bien fonctionné

unique_values_a, counts_a = np.unique(subtrain_y, return_counts=True)
unique_values_b, counts_b = np.unique(subtest_y, return_counts=True)

print(f"Jeu d'entrainement : " + " ".join([f"{value}:{count}" for value, count in zip(unique_values_a, counts_a)]))
print(f"Jeu de test : " + " ".join([f"{value}:{count}" for value, count in zip(unique_values_b, counts_b)]))   

Jeu d'entrainement : 0:3500 1:3500
Jeu de test : 0:1500 1:1500


In [12]:
subtrain_X.shape

(7000, 18725)

# Apprentissage automatique

Nous allons tester différents algorithmes de classification supervisée qui traitent ici les textes en suivant l'hypothèse du sac de mots (BoW).

## Simple régression logistique

## Tâche : Classification avec régression logistique

L'objectif de cette tâche est d'effectuer une classification en utilisant un modèle de régression logistique. Il s'agit d'entraîner le modèle sur un sous-ensemble de données d'entraînement et de prédire les étiquettes sur un sous-ensemble de données de test.

In [13]:
# lr = YOUR Code

lr.fit(subtrain_X, subtrain_y)
pred_y_lr = lr.predict(subtest_X)

In [14]:
print("Ce qui est prédit : " + str(pred_y_lr[0:20]))

print("La vérité : " + str(subtest_y[0:20]))

Ce qui est prédit : [1 1 0 1 1 1 1 0 1 1 1 1 0 1 1 0 1 1 0 1]
La vérité : 1370    1
2240    1
8581    0
2338    1
2929    1
3212    1
1583    1
5728    0
3049    1
7613    0
9233    0
3658    1
5692    0
3413    1
4198    1
128     1
898     1
4264    1
7611    0
505     1
Name: classes, dtype: int64


In [15]:
res_test = np.sum(pred_y_lr == subtest_y) / float(len(subtest_y))
                                       
print(f"Réussite en validation {res_test:.1%}")

Réussite en validation 86.2%


A comparer à la réussite sur le jeu d'entraînement :

In [16]:
pred_y_lr = lr.predict(subtrain_X)

res_train = np.sum(pred_y_lr == subtrain_y) / float(len(subtrain_y))
print(f"Réussite sur les données d'entraînement {res_train:.1%}")

Réussite sur les données d'entraînement 93.6%


Deux remarques importantes à ce stade :

    1. On peut observer une différence entre les deux erreurs, signe d'un sur-apprentissage (*overfitting*)
    2. Nous avons utilisé un même nom de variable (pred_y_lr) pour deux résultats différents : c'est une mauvaise habitude et nous allons utiliser 2 variables différentes pour la suite.

Concernant le sur-apprentissage, une piste serait ici de régulariser le modèle avec un terme qui pénalise une trop grande dispersion des poids (régression ridge et lasso).
Essayons par ex. la régression ridge :

In [17]:
lr_ridge = RidgeClassifier(alpha=20.0)
lr_ridge.fit(subtrain_X, subtrain_y)
pred_train_lr_ridge = lr_ridge.predict(subtrain_X)
pred_test_lr_ridge = lr_ridge.predict(subtest_X)

Le paramètre *alpha* indique la force de la régularisation.

In [18]:
res_train = np.sum(pred_train_lr_ridge == subtrain_y) / float(len(subtrain_y))
res_test = np.sum(pred_test_lr_ridge == subtest_y) / float(len(subtest_y))

print(f"Réussite (accuracy) sur :")
print(f"  - ensemble d'entraînement : {res_train:.1%}")
print(f"  - ensemble de test : {res_test:.1%}")

Réussite (accuracy) sur :
  - ensemble d'entraînement : 88.6%
  - ensemble de test : 83.9%


On constate qu'on diminue le sur-apprentissage, mais au prix d'une réussite plus faible...

## Une évaluation plus robuste : la validation croisée

Il s'agit ici d'évaluation la capacité de l'algorithme choisi à résoudre la tâche de classification.
L'objectif est donc plus de choisir l'algorithme que de trouver le modèle lui-même.

### Tâche

Utilisez la validation croisée par k-fold pour évaluer votre modèle. Créez un objet `KFold` avec les paramètres suivants :
- **n_splits** : 10 (nombre de sous-ensembles dans lesquels les données seront divisées).
- **shuffle** : True (pour mélanger les données avant de les diviser).
- **random_state** : une valeur fixe pour garantir la reproductibilité (utilisez la variable `seed` à cet effet).


In [19]:
seed = 7
np.random.seed(seed)
# kfold = YOUR Code

results = cross_val_score(lr, subtrain_X, subtrain_y, cv=kfold)
print(f"Validation croisée : moyenne {results.mean():.1%} et écart-type {results.std():.2f}")

Validation croisée : moyenne 86.5% et écart-type 0.01


Une fois l'algorithme choisi, il s'agit de réentraîner le modèle sur l'ensemble des données d'entraînement (cf. 3.1).

Pour simplifier le code, on crée une fonction d'affichage des deux erreurs :

In [20]:
def print_accuracy(nom_algo, pred_train, pred_test):
    print(nom_algo + " : ")
    acc_app = np.sum(pred_train == subtrain_y) / float(len(subtrain_y))
    print(f"  - réussite (accuracy) apparente : {acc_app:.1%}")
    acc_gen = np.sum(pred_test == subtest_y) / float(len(subtest_y))
    print(f"  - réussite (accuracy) en généralisation : {acc_gen:.1%}")                                                          

## Machines à vecteurs supports

### Tâche

Implémentez un classificateur SVM (Support Vector Machine) avec un noyau linéaire. Créez un objet `SVC` avec les paramètres suivants :
- **kernel** : "linear" (utilisez un noyau linéaire pour la classification).
- **degree** : 1 (ce paramètre est sans effet pour un noyau linéaire).

In [21]:
svc_lin = SVC(...) # YOUR Code

svc_lin.fit(subtrain_X, subtrain_y)

pred_train_svc_lin = svc_lin.predict(subtrain_X)
pred_test_svc_lin = svc_lin.predict(subtest_X)

print_accuracy("SVC linéaire", pred_train_svc_lin, pred_test_svc_lin)

SVC linéaire : 
  - réussite (accuracy) apparente : 97.6%
  - réussite (accuracy) en généralisation : 87.4%


Créez un classificateur SVM avec un noyau polynomial. Utilisez `SVC` avec les paramètres suivants :
- **kernel** : "poly"
- **degree** : 2


In [22]:
svc_poly2 = SVC(...) # YOUR Code

svc_poly2.fit(subtrain_X, subtrain_y)

pred_train_svc_poly2 = svc_poly2.predict(subtrain_X)
pred_test_svc_poly2 = svc_poly2.predict(subtest_X)

print_accuracy("SVC poly degré 2", pred_train_svc_poly2, pred_test_svc_poly2)

SVC poly degré 2 : 
  - réussite (accuracy) apparente : 99.8%
  - réussite (accuracy) en généralisation : 85.9%


## Arbres de décision

In [23]:
xgb = XGBClassifier(verbosity=0)

xgb.fit(subtrain_X, subtrain_y)

pred_train_xgb = xgb.predict(subtrain_X)
pred_test_xgb = xgb.predict(subtest_X)

print_accuracy("XGBoost", pred_train_xgb, pred_test_xgb)

XGBoost : 
  - réussite (accuracy) apparente : 94.7%
  - réussite (accuracy) en généralisation : 86.0%


## Réseau de neurones simple

In [24]:
mlp = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(3), random_state=1)

mlp.fit(subtrain_X, subtrain_y)

pred_train_mlp = mlp.predict(subtrain_X)
pred_test_mlp = mlp.predict(subtest_X)

print_accuracy("MLP 1 couche cachée", pred_train_mlp, pred_test_mlp)

MLP 1 couche cachée : 
  - réussite (accuracy) apparente : 99.9%
  - réussite (accuracy) en généralisation : 86.6%


# Affichage des résultats

Il faut créer une table qui résume les principaux résultats obtenus.
Ici, on se contentera d'utiliser la pertinence (*accuracy*) des résultats, mais d'autres critères pourraient être employés : temps d'apprentissage, temps pour l'inférence, etc.

In [25]:
#fonction qui retourne les scores d'erreur
def get_accuracy(pred_train, pred_test):
    acc_app = np.sum(pred_train == subtrain_y) / float(len(subtrain_y))
    acc_gen = np.sum(pred_test == subtest_y) / float(len(subtest_y))
    return acc_app, acc_gen

In [26]:
#Calcul des réussites (accuracy)

pred_train_lr = lr.predict(subtrain_X)
pred_test_lr = lr.predict(subtest_X)
acc_train_lr, acc_test_lr = get_accuracy(pred_train_lr, pred_test_lr)
pred_train_lr_ridge = lr_ridge.predict(subtrain_X)
pred_test_lr_ridge = lr_ridge.predict(subtest_X)
acc_train_lr_ridge, acc_test_lr_ridge = get_accuracy(pred_train_lr_ridge, pred_test_lr_ridge)
acc_train_svc_lin, acc_test_svc_lin = get_accuracy(pred_train_svc_lin, pred_test_svc_lin)
acc_train_svc_poly2, acc_test_svc_poly2 = get_accuracy(pred_train_svc_poly2, pred_test_svc_poly2)
acc_train_xgb, acc_test_xgb = get_accuracy(pred_train_xgb, pred_test_xgb)
acc_train_mlp, acc_test_mlp = get_accuracy(pred_train_mlp, pred_test_mlp)

In [27]:
noms_algo = ["Régression logistique", "Classification Ridge", "SVM linéaire", "SVM polynôme 2", "XGBoost", "MLP 1 couche cachée"]
acc_app = [acc_train_lr, acc_train_lr_ridge, acc_train_svc_lin, acc_train_svc_poly2, acc_train_xgb, acc_train_mlp]
acc_gen = [acc_test_lr, acc_test_lr_ridge, acc_test_svc_lin, acc_test_svc_poly2, acc_test_xgb, acc_test_mlp]

In [28]:
res = pd.DataFrame({"algo" : noms_algo, "accuracy app": acc_app, "accuracy gen": acc_gen})

In [29]:
res.style.format({
    "accuracy app": '{:,.2%}'.format,
    "accuracy gen": '{:,.2%}'.format
})

,algo,accuracy app,accuracy gen
0,Régression logistique,93.64%,86.23%
1,Classification Ridge,88.57%,83.93%
2,SVM linéaire,97.60%,87.43%
3,SVM polynôme 2,99.76%,85.87%
4,XGBoost,94.69%,86.00%
5,MLP 1 couche cachée,99.94%,86.57%


On constate ici que le meilleur classifieur est linéaires. Plusieurs explications sont possibles :

- Le problème est très simple et un classifieur linéaire suffit à obtenir de (très) bonnes performances
- L'ensemble de test n'est pas très représentatif et ressemble trop aux données d'entraînement (biais dans l'acquisition des données)

L'écart entre la réussite apparente (ie calculée sur les données d'entraînement) et la réussite en généralisation (ie calculées sur les données test) reflète le phénomène de sur-apprentissage ou *overfitting*.

Cela peut nous aider à mieux mettre au point les classifieurs, mais c'est bien la réussite en généralisation qui guide le classifieur qui sera finalement choisi.